In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Objective**

1) Show the visualisation dashboard with Jupyterdash on Kaggle

2) Use Dash interactivity and Plotly Express Treemap, Line plots to let the audience explore the data

2) Try Jupyter Dash functionality with Kaggle Notebook. It works-

3) Improve on the dashboard, so the design can be used for production purposes

In [ ]:
!pip install dash jupyter_dash dash_core_components dash_table  dash_bootstrap_components dash_html_components 

In [ ]:
import plotly
import plotly.express as px
import plotly.graph_objects as go
import dash
import pandas as pd
import jupyter_dash as jd
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input, State
from dash.exceptions import PreventUpdate
from dash_table import DataTable
pd.options.display.max_columns = None
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')
for p in [plotly, dash, jd, dcc, html, dbc, pd,]:
    print(f'{p.__name__:-<30}v{p.__version__}')

### Using a combined version of Learnplatform engagement dataset.

In [ ]:
product = pd.read_csv('../input/learn-platform-districtengagement/district_engagement.csv',index_col=0,parse_dates=['time'])

In [ ]:
product.sample()

### Again Learn platform Product database

In [ ]:
product_focus = pd.read_csv('../input/learn-platform-districtengagement/product_focus.csv')
product_focus.head()
id_name = product_focus[['LP_ID','Product_Name']]
id_name.head()

In [ ]:
product = pd.merge(left=product,right=id_name,left_on='lp_id',right_on='LP_ID',how='left')
product.head()

In [ ]:
product = product[product.lp_id.isin(id_name.LP_ID)]

In [ ]:
district_modified = pd.read_csv('../input/learn-platform-districtengagement/district_modified.csv')
district_modified.head()

### District information from Learnplatform dataset

In [ ]:
product = product[product.district.isin(district_modified.district_id)]
product = pd.merge(left=product,right=district_modified[['state','district_id']],left_on='district',
                   right_on='district_id',how='left')
product.head()

### After merging the dataset with School District, State and Product, it is ready for visualisation 

In [ ]:
#Plotting treemap chart with one of the states
utah_df = product[product.state == 'Utah']
utah_df.head()
utah_df.loc[:,'district'] = utah_df.district.apply(lambda x: str(x))
# We can create bar chart for each school, pct_access and engagement index
# We can create bar chart for each product
# We can create stacked bar chart for each state, and each product
px.treemap(utah_df, path =['district','Product_Name'],
      values='engagement_index',color='district')

### Firing up the Dash app, using the JupyterDash library. Still exploring ways to create the reports like R  

In [ ]:
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff
from IPython.display import HTML, Image

In [ ]:
#Opening up the big guns, so the data can explored in realtime. 
#Inspired by 
import numpy as np
learnpf=  JupyterDash(__name__, external_stylesheets=[dbc.themes.COSMO])

def make_empty_fig():
    fig = go.Figure()
    fig.layout.paper_bgcolor = '#E5ECF6'
    fig.layout.plot_bgcolor = '#E5ECF6'
    return fig

learnpf.layout = html.Div([
    dbc.Col([
        html.H1('Product Access & Engagement across USA'),
        html.H2('Learn PF Dataset'),

    ], style={'textAlign': 'center'}),
    html.Br(),
    dbc.Row([
        dbc.Col(lg=1),
        dbc.Col([
            dcc.Dropdown(id='product_dropdown',
                        value='Learning A-Z',
                        options=[{'label': lpid, 'value': str(lpid)}
                                for lpid in product.Product_Name.unique()]),
            dcc.Graph(id='product_chart'),
        ], lg=10)
    ]),
    html.Br(),
    html.H2('State and school district access', style={'textAlign': 'center'}),
    html.Br(),
    dbc.Row([
        dbc.Col(lg=1),
        dbc.Col([
            dbc.Label('State'),
            dcc.Dropdown(id='state_dropdown',
                         placeholder='Select a state',
                         options=[{'label': state, 'value': state}
                                  for state in np.sort(product.state.unique())]),
            html.Br(),
            dcc.Graph(id='State_barchart',
                      figure=make_empty_fig())
        ], md=12, lg=5),
        dbc.Col([
            dbc.Label('School'),
            dcc.Dropdown(id='school_dropdown',
                         placeholder='Select School Districts',
                         options=[{'label': school, 'value': school}
                                  for school in np.sort(product.district.unique())]),
            html.Br(),
            dcc.Graph(id='school_timeseries',
                      figure=make_empty_fig())
        ], md=12, lg=5),
    ]),
])
@learnpf.callback(Output('product_chart', 'figure'),
              Input('product_dropdown', 'value'))

def product_chart(lpid):
    lpid_df = product[product.Product_Name == lpid]
    lpid_df.sort_values(by='time',ascending=True,inplace=True)
    lpfig = go.Figure()
    lpfig.add_trace(go.Scatter(x=lpid_df.time,
                              y=lpid_df.engagement_index,
                              mode='lines',
                              line_color='blue',
                              name='engagement_index'))
    lpfig.add_trace(go.Scatter(x=lpid_df.time,
                              y=lpid_df.pct_access,
                              mode='lines',
                              line_color='green',
                              name='Percentage_access'))
    return lpfig

    
@learnpf.callback(Output('State_barchart', 'figure'),
              Input('state_dropdown', 'value'))

def state_chart(state):
    state_df=product[product.state == state]
    state_df.loc[:,'district'] = state_df.district.apply(lambda x: str(x))
    state_fig = px.treemap(state_df, path =[px.Constant("all"),'district','Product_Name'],
                           values='engagement_index',color='district')
    return state_fig
    
@learnpf.callback(Output('school_timeseries', 'figure'),
                  Input('school_dropdown', 'value'))
    
def school_series(school):
    school_df = product[product.district == school]
    school_df.sort_values(by='time',ascending=True,inplace=True)
    grp_school = school_df.groupby('time')['pct_access','engagement_index'].sum().reset_index()
    school_fig = px.line(grp_school,x='time',y='engagement_index')
    return school_fig
    
learnpf.run_server(mode='inline',height=200)